# Parsing Debug Notebook

This somewhat messy notebook makes it easier to debug the parser, because we can just rerun the cells needed to set up the debugging process with various parameters.

In [1]:
import json
import math
import re
from pathlib import Path

import boto3
import pandas as pd

from parse_990_textract.bucket import open_df
from parse_990_textract.filing import create_roadmap, extract_from_roadmap
from parse_990_textract.models import BoundingBox, TableExtractor
from parse_990_textract.parse import create_extractors, find_item, find_pages
from parse_990_textract.postprocessing import clean_filing, clean_f_i, clean_f_ii, clean_f_iii, postprocess
from parse_990_textract.setup import load_extractor_df
from parse_990_textract.table import extract_table_data, find_table_pages, create_tablemap
from parse_990_textract.utils import get_coordinate, get_regex, cluster_words, columnize, cluster_x

In [2]:
bucket = boto3.resource("s3").Bucket("s3-ocr-990s-demo")

In [3]:
validation_data = pd.read_csv("validation_data.csv", index_col="job_id").fillna("")
validation_data.head()

,pdf_key,other_expenses_c_mgmt_general,payments_affiliates_total,other_expenses_d_prog_service,total_revenue,fees_for_lobbying_services_total,other_employee_benefits_fundraising,total_fundraising_expenses,compensation_officers_prog_service,fees_for_mgmt_services_total,...,travel_total,pension_plan_acc_contrib_prog_service,advertising_promotion_mgmt_general,pension_plan_acc_contrib_mgmt_general,compensation_disq_persons_total,other_salaries_wages_mgmt_general,office_expenses_mgmt_general,fees_for_other_services_fundraising,total_number_other_recipient_foreign_orgs_entities,activities_per_region_totals_total_expenditure
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,,,33000.0,1415024,,,4503.0,,,...,18307.0,,,,,,,,,
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,,,15832.0,9899921,,27161.0,597789.0,295722.0,,...,677722.0,34261.0,30,3462.0,,159156.0,48381.0,22350.0,,
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,0.0,,10823.0,3370595,,12776.0,263765.0,148689.0,2250.0,...,197092.0,,127381,,396855.0,103312.0,11723.0,23021.0,,
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,,,,46117,,,669.0,,,...,,,,,,,458.0,,,
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581.0,,0.0,583209,,2521.0,14409.0,298484.0,,...,7233.0,,,,,,4010.0,0.0,,


In [4]:
extractor_df = load_extractor_df("990_extractors.csv")
roadmap_df = load_extractor_df("990_roadmap.csv")
schedule_f_tablemap_df = load_extractor_df("schedule_f_table_roadmap.csv")
schedule_f_table_extractor_df = pd.read_csv("schedule_f_table_extractors.csv")
schedule_f_row_extractor_df = pd.read_csv("schedule_f_row_extractors.csv")

In [5]:
PART_I_HEADER = r"\(a\)\s*Region|\(d\)\s*Activities|\(e\)\s*If activity|\(f\)Total expenditures"
PART_II_HEADER = r"\(b\)\s*IRS code|\(c\)\s*Region|\(d\)\s*Purpose|\(f\)\s*Manner|\(h\)\s*Description"
PART_III_HEADER = r"\(b\)\s*Region|\(e\)\s*Manner of cash|\(h\)\s*Method of va"
PART_I_TABLE_NAME = r"Activities per Region"
PART_II_TABLE_NAME = r"Grants to Organizations Outside the United States"
PART_III_TABLE_NAME = r"Grants to Individuals Outside the United States"

In [6]:
filing_rows = []
schedule_f_part_i_rows = []
schedule_f_part_ii_rows = []
schedule_f_part_iii_rows = []


for i, job_id in enumerate(validation_data.index.values):
    print(i)
    print(job_id)
    pdf_key = validation_data.at[job_id, "pdf_key"]
    print(pdf_key)
    
    data = open_df(bucket, job_id)
    lines = data.loc[data["BlockType"] == "LINE"]
    words = data.loc[data["BlockType"] == "WORD"]
    page_map = find_pages(lines)
    roadmap = create_roadmap(
        lines, roadmap_df, page_map
    )
    
    row = extract_from_roadmap(
        words, lines, roadmap, extractor_df, page_map
    )
    row = postprocess(row, job_id, pdf_key, clean_filing)
    filing_rows.append(row)
    
    pages = lines.groupby("Page")
    
    part_i_table = extract_table_data(
        pages, lines, words, PART_I_HEADER, PART_I_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_i_table = postprocess(part_i_table, job_id, pdf_key, clean_f_i)
    if part_i_table is not None:
        schedule_f_part_i_rows.append(
            part_i_table
        )
    part_ii_table = extract_table_data(
        pages, lines, words, PART_II_HEADER, PART_II_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_ii_table = postprocess(part_ii_table, job_id, pdf_key, clean_f_ii)
    if part_ii_table is not None:
        schedule_f_part_ii_rows.append(
            part_ii_table
        )
    part_iii_table = extract_table_data(
        pages, lines, words, PART_III_HEADER, PART_III_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_iii_table = postprocess(part_iii_table, job_id, pdf_key, clean_f_iii)
    if part_iii_table is not None:
        schedule_f_part_iii_rows.append(
            part_iii_table
        )

0
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4
EIN_760733035_YEAR_2009_FORMTYPE_990.pdf


No match for year_formation in L Year of Formation M State of legal domicile
No match for state_of_domicile in M State of legal domicile


1
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc
EIN_363235550_YEAR_2009_FORMTYPE_990.pdf


No match for total_unrelated_biz_revenue in 7a NONE
No match for net_unrelated_biz_taxable_revenue in 7b NONE
No match for benefits_paid_members_expenses in NONE
No match for professional_fundraising_fees_expenses in NONE
No match for grants_foreign_individuals_govt_orgs_total in NONE
No match for benefits_to_members_total in NONE
No match for compensation_disq_persons_total in NONE
No match for fees_for_mgmt_services_total in NONE
No match for fees_for_lobbying_services_total in NONE
No match for fees_for_fundraising_services_total in NONE
No match for fees_for_investment_mgmt_services_total in NONE
No match for royalties_total in NONE
No match for entertainment_travel_govt_officials_total in NONE
No match for interest_total in NONE
No match for payments_affiliates_total in NONE


2
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b
EIN_223134995_YEAR_2010_FORMTYPE_990.pdf
3
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82
EIN_264320885_YEAR_2009_FORMTYPE_990.pdf
4
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be
EIN_581943161_YEAR_2009_FORMTYPE_990.pdf
5
5596f51a999ebbd4cb992f490ceaffcddbac9bce532b911997683ae6897c2797
EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
6
481a62c75776cc7f62075c1d60c230ea01f2dbc5b6224c65cb73db1728e86b00
EIN_954806856_YEAR_2009_FORMTYPE_990.pdf
7
39d270117e4b6354850551c8237421403da7d2b3f5d06657a28f5a9d07febf17
EIN_521238301_YEAR_2010_FORMTYPE_990.pdf


...

No match for activities_per_region_totals_number_of_offices in O


19
aefc7b65c34db330d8d9f56a1226e116b63ee9be7dbdd4ae4c7bea5d87359f97
EIN_362428692_YEAR_2009_FORMTYPE_990.pdf
20
a5a3cbfcf844be8862bbb61ad46d4c795891ab1143e420db5ed99fc79eeb66c9
EIN_271377148_YEAR_2016_FORMTYPE_990.pdf
21
67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebdb32d3186b19d6a6fd5
EIN_42103580_YEAR_2010_FORMTYPE_990.pdf
22
a6529e504df346097da99104a353e977426e018cd5ac33b62cd2dd89c90763c5
EIN_311002913_YEAR_2008_FORMTYPE_990.pdf
23
d1925c2d74adaa3f150ded3ce67dfe7ae3a306f0db0289ad5755a28d801b2b0b
EIN_202408857_YEAR_2010_FORMTYPE_990.pdf


No match for activities_per_region_totals_number_of_offices in Act the


24
01d89ee5d14575c1321b2e4d67431d172ba76212b4a266bdaf474275029fd78b
EIN_521830327_YEAR_2009_FORMTYPE_990.pdf


No match for total_fundraising_expenses in b Total fundraising expenses (Part IX, column (D), line 25)
No match for revenue_less_expenses in <43,775.> End of Year


In [7]:
output_data = pd.concat(filing_rows).reset_index(drop=True).set_index("job_id")

In [8]:
output_data.head()

field_name,index,name,address,city,state,zip,website,gross_receipts,year_formation,state_of_domicile,...,total_functional_expense_fundraising,activities_per_region_totals_number_of_offices,activities_per_region_totals_number_of_employees,activities_per_region_totals_total_expenditure,total_number_recipient_foreign_orgs_listed_as_charities,total_number_other_recipient_foreign_orgs_entities,pdf_key,ein,year,filing_id
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,0,MAKE WAY PARTNERS INC,PO BOX 26367,BIRMINGHAM,OX,26367,www MAKEWAYPARTNERS ORG,1426217,,,...,4503,,,,,,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,760733035,2009,760733035_2009
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,0,THE FEDERALIST SOCIETY FOR LAW AND,"1015 18TH ST., N.W. 425",WASHINGTON,DC,20036,www. FED-SOC.ORG,11033302,1982,IL,...,597789,,,,,,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,363235550,2009,363235550_2009
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,0,Ron Hutchcraft Ministries Inc,PO Box 400,Harnson,AR,20400,www hutchcraft com,3390,1991,NJ,...,263765,,,,,,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,223134995,2010,223134995_2010
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,0,FREEDOM 4 24,21430 TIMBERLAKE ROAD STE 101,LYNCHBURG,VA,24502,freedom424 org,61857,2009,VA,...,669,,,,,,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,264320885,2009,264320885_2009
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,0,GEORGIA PUBLIC POLICY FOUNDTION,6100 LAKE FORREST DR,LAKE FORREST,GA,30328,www GPPF org,583209,1991,GA,...,14409,,,,,,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581943161,2009,581943161_2009


In [9]:
def clean(x):
    x = str(x)
    x = re.sub(r"\.0\b", "", x)
    x = re.sub("\D", "", x)
    return x

In [10]:
def compare_output(to_validate, to_compare, col):
    return pd.DataFrame(
        {
            "extracted": to_validate.loc[col].loc[
                lambda series: series != to_compare.loc[col]
            ],
            "expected": to_compare.loc[col].loc[
                lambda series: series != to_validate.loc[col]
            ],
        }
    )

In [11]:
to_compare = validation_data.set_index("pdf_key").applymap(clean)
to_validate = output_data[validation_data.columns].set_index("pdf_key").applymap(clean)

for col in to_validate.index:
    validated = compare_output(to_validate, to_compare, col)
    if validated.any().any():
        print(col)
        print(f"{validated.shape[0]} mismatched items.")
        print(validated)
        print("-"*79)

EIN_113489123_YEAR_2008_FORMTYPE_990.pdf
2 mismatched items.
              extracted expected
total_revenue      1017  1017506
travel_total        137    13710
-------------------------------------------------------------------------------
EIN_582248383_YEAR_2009_FORMTYPE_990.pdf
1 mismatched items.
                                                   extracted expected
total_number_other_recipient_foreign_orgs_entities        10         
-------------------------------------------------------------------------------


In [12]:
assert False

AssertionError: 

## BREAK

In [13]:
pd.concat(schedule_f_part_i_rows).head(50)[
    [
        "region", "number_offices", "number_employees", 
        "activities_conducted", "specific_type_activity", 
        "total_expenditures", "pdf_key"
    ]
]

field,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,pdf_key
0,efile GRAPHIC SCHEDULE F (Form 990),,,As Filed Activities,,,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
1,Department of the Treasury Internal Revenue S...,,,"Complete Form 990,",,,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
2,"Part I General ""Yes"" 1 For grantmakers. assist...",,,Outside grants or,,,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
3,2 For grant makers. United States,,,,,,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
4,3 Activites per (a) Region,,,990) if of or in region,,,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
5,Europe,,1,0,,,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
6,Totals,,1,o,,,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
0,Schedule F (Form 990) Department of the Treas...,,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
1,"AMERICAN LIFE Part I General to Form 990, 1 Fo...",,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
2,2 For grantmakers.,,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf


In [14]:
pd.concat(schedule_f_part_i_rows).tail(50)

field,index,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,job_id,pdf_key,ein,year,filing_id
3,3,SUB-SAHARAN,,,,,,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...,EIN_582248383_YEAR_2009_FORMTYPE_990.pdf,582248383,2009,582248383_2009
4,4,EUROPE,,,,,,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...,EIN_582248383_YEAR_2009_FORMTYPE_990.pdf,582248383,2009,582248383_2009
5,5,Totals,,,,,,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...,EIN_582248383_YEAR_2009_FORMTYPE_990.pdf,582248383,2009,582248383_2009
0,0,efile GRAPHIC print - DO SCHEDULE F (Form 990...,,,"- Outside the ""Yes"" to Form 990, Part to For...",,,68a8d4678de1d3107eff3ae0bfa1acdd6a9787a173b49c...,EIN_474865647_YEAR_2020_FORMTYPE_990.pdf,474865647,2020,474865647_2020
1,1,"Part I General Information Form 990, Part",,,United States.,,,68a8d4678de1d3107eff3ae0bfa1acdd6a9787a173b49c...,EIN_474865647_YEAR_2020_FORMTYPE_990.pdf,474865647,2020,474865647_2020
2,2,"1 For grantmakers. Does other assistance, the ...",,,"to substantiate the assistance, and the",,,68a8d4678de1d3107eff3ae0bfa1acdd6a9787a173b49c...,EIN_474865647_YEAR_2020_FORMTYPE_990.pdf,474865647,2020,474865647_2020
3,3,2 For grantmakers. Describe outside the United...,,,procedures for monitoring,,,68a8d4678de1d3107eff3ae0bfa1acdd6a9787a173b49c...,EIN_474865647_YEAR_2020_FORMTYPE_990.pdf,474865647,2020,474865647_2020
4,4,3 Activites per Region. (The (a) Region S...,,,duplicated if additional of (d) Activities con...,,,68a8d4678de1d3107eff3ae0bfa1acdd6a9787a173b49c...,EIN_474865647_YEAR_2020_FORMTYPE_990.pdf,474865647,2020,474865647_2020
0,0,efile GRAPHIC print SCHEDULE F (Form 990) D...,,,As Filed Activities if the Part to Form,,,2ef32905e24a7a69d5bb4e4ac22448b279cbe84df831d5...,EIN_262414132_YEAR_2009_FORMTYPE_990.pdf,262414132,2009,262414132_2009
1,1,"Part I General ""Yes"" to 1 For grantmakers. ass...",,,Outside grants,,,2ef32905e24a7a69d5bb4e4ac22448b279cbe84df831d5...,EIN_262414132_YEAR_2009_FORMTYPE_990.pdf,262414132,2009,262414132_2009


In [ ]:
pd.concat(schedule_f_part_ii_rows).iloc[:50]

In [ ]:
pd.concat(schedule_f_part_iii_rows).iloc[:50]

In [15]:
table_test_df = open_df(bucket, "5596f51a999ebbd4cb992f490ceaffcddbac9bce532b911997683ae6897c2797")

In [16]:
test_lines = table_test_df.loc[
    table_test_df["BlockType"] == "LINE"
]
test_words = table_test_df.loc[
    table_test_df["BlockType"] == "WORD"
]
test_pages = test_lines.groupby("Page")

In [17]:
HEADER = PART_I_HEADER
NAME = PART_I_TABLE_NAME

In [18]:
table_pages = find_table_pages(
    test_pages["Text"].agg(lambda words: " ".join(words)),
    HEADER,
)

In [19]:
table_pages

Page
22    22
Name: Page, dtype: int64

In [20]:
TEST_INDEX = 0
TEST_PAGE = 22

In [21]:
tablemaps = pd.DataFrame(
    {
        "page": table_pages,
        "tablemap": table_pages.map(
            lambda page: create_tablemap(
                test_lines, schedule_f_tablemap_df, page, NAME
            ).dropna()
        ),
    }
)

In [22]:
tablemaps["tablemap"].iloc[TEST_INDEX]

,Top,Left,Top_Default,Left_Default
Item,,,,
(a) Region,0.335665,0.045264,0.05,0.0
(b) Number of offices,0.362746,0.205227,0.05,0.0
(c) Number of employees,0.363006,0.311407,0.05,0.0
(d) Activities conducted,0.352594,0.420936,0.05,0.0
(e) Specific type,0.355577,0.572975,0.05,0.0
(f) Total Expenditures,0.370040,0.735998,0.05,0.0
"Schedule F, Part I, Item 3a",0.988474,0.007641,0.00,0.0
Bottom (d),0.401869,0.459520,0.00,0.0
Bottom Part I,0.988482,0.725221,0.75,0.0


In [23]:
row_extractors = schedule_f_row_extractor_df.loc[
    schedule_f_row_extractor_df["table"] == NAME
]
table_data = schedule_f_table_extractor_df.loc[
    schedule_f_table_extractor_df["table"] == NAME
].iloc[0]

In [24]:
rows = tablemaps.assign(
    extractor=tablemaps["tablemap"].map(
        lambda tablemap: TableExtractor(
            header_top_label=table_data["header_top"],
            top_label=table_data["table_top"],
            bottom_label=table_data["table_bottom"],
            tablemap=tablemap,
            fields=row_extractors["field"],
            field_labels=row_extractors["col_left"]
        )
    )
)

In [25]:
test_extractor = rows["extractor"].iloc[TEST_INDEX]

In [26]:
test_extractor.get_col_spans(test_words, TEST_PAGE)

0                     (0.0, 0.1110638976097107)
1     (0.1110638976097107, 0.24223646521568298)
2    (0.24223646521568298, 0.33819979429244995)
3      (0.33819979429244995, 0.572974681854248)
4       (0.572974681854248, 0.7359976768493652)
5                     (0.7359976768493652, 1.0)
dtype: object

In [27]:
test_extractor.field_labels

0             Top Left Corner
1       (b) Number of offices
2     (c) Number of employees
3    (d) Activities conducted
4           (e) Specific type
5      (f) Total Expenditures
Name: col_left, dtype: object

In [28]:
test_extractor.extract_rows(test_words, TEST_PAGE).head(50)

field,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures
0,efile GRAPHIC SCHEDULE F (Form 990),print - DO NOT Statement,PROCESS of,As Filed Activities,,
1,Department of the Treasury Internal Revenue S...,Attach THEOLOGICAL SEMINARY,to Form 990.,"Complete Form 990,",,
2,"Part I General ""Yes"" 1 For grantmakers. assist...","Information to Form 990, Part Does the the gra...","on Activities IV, line 14b. organization maint...",Outside grants or,,
3,2 For grant makers. United States,Describe in Part,IV the organization's,,,
4,3 Activites per (a) Region,Region (Use Schedule (b) Number offices region,F-1 (Form of (c) Number in the employees agents,990) if of or in region,,
5,Europe,,1,0,,
6,Totals,,1,o,,


In [29]:
table_words = test_extractor.get_table_words(test_words, TEST_PAGE)

In [30]:
table_words

,BlockType,Confidence,Geometry,Page,PageClassification,Relationships,Text,TextType,Polygon,Height,Left,Top,Right,Bottom,Midpoint_X,Midpoint_Y,Width,Children,Line_No,File
Id,,,,,,,,,,,,,,,,,,,,
0370b40f-1998-47e6-bda8-d1568a901fdb,WORD,99.867920,{'BoundingBox': {'Height': 0.01207768637686967...,22,None,None,efile,PRINTED,"[{'X': 0.013244425877928734, 'Y': 0.0134243108...",0.012078,0.013244,0.013424,0.048931,0.025502,0.031088,0.019463,0.035687,None,1,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...
06409b4c-abdd-48ac-8b16-4988c6f6115f,WORD,99.119987,{'BoundingBox': {'Height': 0.01237831357866525...,22,None,None,GRAPHIC,PRINTED,"[{'X': 0.05483809486031532, 'Y': 0.01336579117...",0.012378,0.054838,0.013366,0.126745,0.025744,0.090791,0.019555,0.071907,None,1,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...
0e285100-f5b7-4c52-99ea-2c5f241d298e,WORD,99.878563,{'BoundingBox': {'Height': 0.01385955605655908...,22,None,None,print,PRINTED,"[{'X': 0.13135938346385956, 'Y': 0.01356557663...",0.013860,0.131359,0.013566,0.171309,0.027425,0.151334,0.020495,0.039949,None,1,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...
62405d05-abd8-4231-a989-cc374b46d3ee,WORD,99.687042,{'BoundingBox': {'Height': 0.01174192037433385...,22,None,None,DO,PRINTED,"[{'X': 0.18775515258312225, 'Y': 0.01367457304...",0.011742,0.187755,0.013675,0.209762,0.025416,0.198759,0.019546,0.022007,None,1,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...
c9adbbf1-d8f9-49cf-9d2d-304ef7c5525c,WORD,99.912117,"{'BoundingBox': {'Height': 0.0118419099599123,...",22,None,None,NOT,PRINTED,"[{'X': 0.21500122547149658, 'Y': 0.01368432957...",0.011842,0.215001,0.013684,0.247244,0.025526,0.231122,0.019605,0.032243,None,1,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
add9f851-1d2b-493a-9d27-a5e7b8ff1edf,WORD,99.557724,{'BoundingBox': {'Height': 0.00941968057304620...,22,None,None,1,PRINTED,"[{'X': 0.28770220279693604, 'Y': 0.41764032840...",0.009420,0.287702,0.417640,0.294043,0.427060,0.290872,0.422350,0.006340,None,48,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...
24dd46cd-a6d4-4d73-bbde-8bcb1bd82372,WORD,51.695694,{'BoundingBox': {'Height': 0.00932173337787389...,22,None,None,0,PRINTED,"[{'X': 0.39219629764556885, 'Y': 0.41763320565...",0.009322,0.392196,0.417633,0.399711,0.426955,0.395954,0.422294,0.007515,None,48,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...
5bc018ea-6cb8-404e-bb6d-05ed91de6570,WORD,99.459747,{'BoundingBox': {'Height': 0.01006298139691352...,22,None,None,Totals,PRINTED,"[{'X': 0.006998767144978046, 'Y': 0.9649543166...",0.010063,0.006999,0.964954,0.049642,0.975017,0.028321,0.969986,0.042644,None,97,5596f51a999ebbd4cb992f490ceaffcddbac9bce532b91...


In [ ]:
word_clusters = cluster_words(table_words, table_words["Height"].min(), "Midpoint_Y")
[" ".join(word.sort_values(by="Left")["Text"].values) for word in word_clusters]

In [ ]:
def columnize(word_cluster, col_spans):
    return col_spans.map(
        lambda span: word_cluster.loc[
            (word_cluster["Left"].between(*span, inclusive="left"))
        ]
    )


def get_cluster_coords(cluster):
    cluster_coords = {
        "Left": cluster["Left"].min(),
        "Right": cluster["Right"].max(),
        "Height": cluster["Height"].max(),
        "Midpoint_X": cluster["Midpoint_X"].median(),
        "Midpoint_Y": cluster["Midpoint_Y"].median(),
        "Top": cluster["Top"].min(),
        "Bottom": cluster["Bottom"].min(),
    }
    cluster_coords["Width"] = cluster_coords["Right"] - cluster_coords["Left"]
    return cluster_coords


def combine_row(row):
    return pd.Series([
        line.map(
            lambda x: x.sort_values(
                by="Left"
            ).reset_index(drop=True)["Text"].fillna("")
        ).agg(
            lambda x: " ".join(x.values)
        ) + " "
        for line in row
    ]).sum().str.strip()

col_spans = test_extractor.get_col_spans(test_words, TEST_PAGE)

col_spans

In [ ]:
y_tol = table_words["Height"].max() * 2
x_tol = table_words["Width"].median()
sum_y_delta = y_tol
columnized = columnize(word_clusters[0], col_spans)
last_col_coords = pd.DataFrame.from_records(
    columnized.map(
        get_cluster_coords
    )
)
print(y_tol)
rows = []
current_row = [columnized]
print(" ".join(word_clusters[0].sort_values(by="Left")["Text"].values))
for count, cluster in enumerate(word_clusters[1:]):
    print("-"*50)
    print(" ".join(cluster.sort_values(by="Left")["Text"].values))
    columnized = columnize(cluster, col_spans)
    col_coords = pd.DataFrame.from_records(columnized.map(get_cluster_coords))
    nonempty = col_coords.dropna().index.to_series()
    last_nonempty = last_col_coords.dropna().index.to_series()
    delta_cols = (~nonempty.isin(last_nonempty)).any()
    y_delta = (
        col_coords["Midpoint_Y"].median()
        - last_col_coords["Midpoint_Y"].median()
    )
    sum_y_delta += y_delta
    mean_y_delta = sum_y_delta / (count + 2)
    min_y_delta = min(mean_y_delta, y_tol)
    print("Y Delta:", y_delta)
    print("Min Y Delta:", min_y_delta)
    print("More cols detected", delta_cols and (y_delta > min_y_delta))
    print("Y tolerance exceeded", (y_delta > y_tol) and (y_delta > min_y_delta))
    if (delta_cols or (y_delta > y_tol)) and (y_delta > min_y_delta):
        combined_row = combine_row(current_row)
        print(combined_row)
        rows.append(combined_row)
        current_row = [columnized]
    else:
        current_row.append(columnized)
    last_col_coords = col_coords
print("-"*50)
combined_row = combine_row(current_row)
print(combined_row)
rows.append(combined_row)

In [ ]:
test_lines.loc[
    test_lines["Text"].str.contains("other\)")
    & test_lines["Page"].between(26, 26)
]

In [ ]:
test_extractor.get_table_bottom(test_words, TEST_PAGE)

In [ ]:
test_words.loc[
    test_words["Text"].str.match("579")
    & test_words["Page"].between(TEST_PAGE, TEST_PAGE)
]

In [ ]:
create_tablemap(
    test_lines,
    schedule_f_tablemap_df,
    26,
    NAME
)

In [ ]:
test_extractor.tablemap

In [ ]:
header_words = test_extractor.get_header_words(test_words, 22)

In [ ]:
header_words.sort_values(by="Left")[["Text", "Left", "Right", "Midpoint_Y"]].head(50)

In [ ]:
pd.DataFrame(
    {
        "Gap": header_words.sort_values(by="Left").rolling(2).apply(lambda x: x),
        "Right": header_words.sort_values(by="Left")["Right"],
        "Text": header_words.sort_values(by="Left")["Text"],
    }
)

In [ ]:
def cluster_x(words, tolerance):
    print("Tolerance", tolerance)
    if (tolerance == 0) or (words.shape[0] < 2):
        return [
            [word] for (idx, word)
            in words.sort_values(by="Left").iterrows()
        ]
    groups = []
    sorted_words = words.sort_values(by="Left")
    current_group = [sorted_words.iloc[0]]
    last = sorted_words.iloc[0]["Right"]
    for idx, word in sorted_words.iloc[1:].iterrows():
        if word["Left"] <= (last + tolerance):
            current_group.append(word)
        else:
            print("New Group")
            print("*"*100)
            groups.append(current_group)
            current_group = [word]
        print("Left:", word["Left"])
        print("Right:", word["Right"])
        last = max((last, word["Right"]))
    groups.append(current_group)
    return [pd.DataFrame(group) for group in groups]

In [ ]:
left_right = pd.DataFrame(
    {
        "Left": header_words["Left"].sort_values().reset_index(drop=True),
        "Right": header_words["Right"].sort_values().reset_index(drop=True),
    }
)

header_lines = cluster_x(
    left_right, header_words["Width"].min()*.8
)

In [ ]:
len(header_lines)

In [ ]:
[
    (x["Left"].min(), x["Right"].max())
    for x in header_lines
]

In [ ]:
x_clusters = cluster_x(header_words, header_words["Width"].min()*.8)

In [ ]:
len(x_clusters)

In [ ]:
0.6793086230754852-0.6723970174789429

In [ ]:
[
    words.sort_values(by="Left")["Text"].agg(lambda x: " ".join(x.values))
    for words in x_clusters
]

In [ ]:
left_bounds = pd.Series(
        cluster["Left"].min() for cluster in x_clusters[1:]
)
right_bounds = pd.Series(
    [cluster["Right"].max() for cluster in x_clusters[:-1]]
)
offsets = right_bounds - left_bounds
full_left = pd.concat([pd.Series([0]), left_bounds + offsets]).reset_index(drop=True)
full_right = pd.concat([right_bounds - offsets, pd.Series([1])]).reset_index(drop=True)
col_spans = full_left.combine(full_right, lambda x, y: (x, y))
col_spans

In [ ]:
last_cluster_right = pd.concat([header_words, table_words])["Left"].min()
left_bounds = []
right_bounds = []
for cluster in x_clusters:
    print("Last Cluster Right", last_cluster_right)
    print("Cluster Left", cluster["Left"].min())
    print("Cluster Right", cluster["Right"].max())
    print("*"*50)
    left_bounds.append(last_cluster_right)
    last_cluster_right = max(
        cluster["Right"].max()
        + cluster["Left"].min()
        - last_cluster_right,
        cluster["Right"].max()
    )
    right_bounds.append(last_cluster_right)
col_spans = pd.Series(zip(left_bounds, right_bounds))
col_spans

In [ ]:
header_words["Midpoint_X"].round(2).value_counts()

In [ ]:
def count_crossing_lines(df, left, right):
    return (
        df.loc[
            df["Left"].between(left, right)
            & (df["Right"] > right),
            "Text"
        ].count()
        + df.loc[
            (df["Left"] < left)
            & df["Right"].between(left, right),
            "Text"
        ].count()
    )


def recalculate_intervals(left_interval, right_interval):
    left_left = left_interval[0]
    right_right = right_interval[1]
    total_span = right_right - left_left
    
    

In [ ]:
col_spans.map(
    lambda x: (x[1] + x[0]) / 2
)

In [ ]:
table_words.loc[
    table_words["Left"].between(0.12358373403549194, 0.3558424413204193)
    & (table_words["Midpoint_X"] > 0.3558424413204193),
    "Text"
]

In [ ]:
left_right = pd.DataFrame(
    {
        "left": header_words["Left"].sort_values().reset_index(drop=True),
        "right": header_words["Right"].sort_values().reset_index(drop=True),
    }
)
left_right.rolling(2).apply(lambda x: print(x, "\n" + "*"*20) or 1)

In [ ]:
left_right.head()

In [ ]:
1 and 2

In [ ]:
(test_words.groupby("Page")["Width"].max() / test_words.groupby("Page")["Height"].mean()).index.values

In [ ]:
pd.cut(header_words["Right"], 1000).unique().categories

In [ ]:
sorted_header_words = header_words.sort_values(by="Left")

In [ ]:
len(x_clusters)

In [ ]:
def cluster_x(words, tolerance):
    if (tolerance == 0) or (words.shape[0] < 2):
        return [
            [word] for (idx, word)
            in words.sort_values(by="Midpoint_X").iterrows()
        ]
    groups = []
    sorted_words = words.sort_values(by="Midpoint_X")
    current_group = [sorted_words.iloc[0]]
    last = sorted_words.iloc[0]["Midpoint_X"]
    for idx, word in sorted_words.iloc[1:].iterrows():
        last = (last + word["Midpoint_X"]) / 2
        if word["Midpoint_X"] <= (last + tolerance):
            current_group.append(word)
        else:
            print("New Group")
            print("*"*100)
            groups.append(current_group)
            current_group = [word]
            last = word["Right"]
        print("Word:", word["Text"])
        print("Left:", word["Left"])
        print("Right:", word["Right"])
    groups.append(current_group)
    return [pd.DataFrame(group) for group in groups]

In [ ]:
clusters = cluster_x(header_words, header_words["Width"].mean()*.7)

In [ ]:
len(clusters)

In [ ]:
header_words.sort_values(by="Midpoint_X").rolling(4)["Midpoint_X"].mean()

In [ ]:
left_clusters = cluster_words(header_words, header_words["Width"].mean()*.835, "Left")

In [ ]:
len(left_clusters)

In [ ]:
[
    cluster["Left"].mean() for cluster in left_clusters
]

In [ ]:
def get_col_spans(self, words, page):
    pass

In [ ]:
test_extractor.tablemap

In [ ]:
test_col_spans = pd.Series(
    [
        (0, 0.205227),
        (0.205227, 0.311407),
        (0.311407, 0.420936),
        (0.420936, 0.572975),
        (0.572975, 0.735998),
        (0.735998, 1),
    ]
)

In [ ]:
def find_new_right(df, right):
    return (
        df.loc[
            (df["Right"] > right*1.01)
            & (df["Left"] < right),
            "Left"
        ].min()
    )

In [ ]:
crossing_right = test_col_spans.map(
    lambda x: get_new_right(
        pd.concat([
            test_extractor.get_header_words(test_words, 18),
            test_extractor.get_table_words(test_words, 18),
        ]),
        x[1]
    )
)
crossing_right

In [ ]:
test_col_spans.where(
    crossing_right.isna(),
    test_col_spans.combine(crossing_right, lambda x, y: (x[0], y))
)

In [ ]:
test_col_spans

In [ ]:
init_left = test_extractor.field_labels.map(
    lambda x: get_coordinate(test_extractor.tablemap, x, "Left", "Left_Default")
)
init_right = pd.concat(
    [
        init_left.iloc[1:],
        pd.Series([1]),
    ],
    ignore_index=True,
)

In [ ]:
crossing_right = init_right.map(
    lambda x: find_new_right(
        pd.concat([
            test_extractor.get_header_words(test_words, 25),
            test_extractor.get_table_words(test_words, 25),
        ]),
        x
    )
)
crossing_right

In [ ]:
new_right = init_right.where(
    crossing_right.isna(),
    crossing_right,
)
new_right

In [ ]:
init_left

In [ ]:
init_left.iloc[1:] = new_right.iloc[:-1]

In [ ]:
init_left

In [ ]:
col_spans = init_left.combine(new_right, lambda x, y: (x, y))

In [ ]:
test_lines.loc[
    test_lines["Text"].str.contains("\(a\)\s*Na")
]